In [1]:
import os
import sys

import jax
import jax.numpy as jnp
import numpy as np
import optax
from genot.models.model import GENOT
from genot.nets.nets import MLP_vector_field
import ott
import torch
import seaborn as sns
import genot
import jax
import ott
import diffrax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from genot.nets.nets import MLP_vector_field, MLP_marginal
import sklearn.preprocessing as pp
#import scanpy as sc
from ott.solvers.linear import sinkhorn, acceleration
from genot.data.data import MixtureNormalSampler
from genot.plotting.plots import plot_1D_unbalanced
from eot_benchmark.gaussian_mixture_benchmark import (
    get_guassian_mixture_benchmark_ground_truth_sampler,
    get_guassian_mixture_benchmark_sampler,
    get_test_input_samples,
)
from eot_benchmark.metrics import calculate_cond_bw, compute_BW_UVP_by_gt_samples

/home/icb/leon.stadelmann/miniconda3/envs/eot_benchmark/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-11 12:08:15.263282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733915296.121353 1854478 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733915296.510641 1854478 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import eot_benchmark.gaussian_mixture_benchmark

In [3]:
torch.cuda.get_device_name()

'NVIDIA H100 80GB HBM3'

In [4]:
torch.__version__

'2.5.1+cu124'

In [5]:
class Loader:

    def __init__(self, sampler, batch_size):
        self.sampler = sampler
        self.batch_size = batch_size

    def __next__(self):
        return jnp.asarray(self.sampler.sample(self.batch_size))#.cpu())

In [6]:
out_dir = "./"

arguments = sys.argv
DIM = 64  # choose according to benchmark
EPS = 1.0  # choose according to benchmark
if EPS > 0.1:
    EPS = int(EPS)
BATCH_SIZE = 2048
K_NOISE_PER_X = 1
ITERATIONS = 100000#100_000
LR = 1e-5
SEED = 0


iters = ITERATIONS  # this is just to log a few results

NUM_SAMPLES_cBVP = 1_000
NUM_SAMPLES_BVP = 100_000
GPU_DEVICE = 0

In [7]:
input_sampler = get_guassian_mixture_benchmark_sampler(
    input_or_target="input",
    dim=DIM,
    eps=EPS,
    batch_size=BATCH_SIZE,
    device=f"cuda:{GPU_DEVICE}",
    download=True,
)

target_sampler = get_guassian_mixture_benchmark_sampler(
    input_or_target="target",
    dim=DIM,
    eps=EPS,
    batch_size=BATCH_SIZE,
    device=f"cuda:{GPU_DEVICE}",
    download=True,
)

ground_truth_plan_sampler = get_guassian_mixture_benchmark_ground_truth_sampler(
    dim=DIM, 
    eps=EPS, 
    batch_size=BATCH_SIZE, 
    device=f"cuda:{GPU_DEVICE}", 
    download=True
)

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1HNXbrkozARbz4r8fdFbjvPw8R74n1oiY 

/home/icb/leon.stadelmann/miniconda3/envs/eot_benchmark/lib/python3.9/site-packages/eot_benchmark-1.0-py3.9.egg/eot_benchmark/gaussian_mixture_benchmark.py:196: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mo

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1HNXbrkozARbz4r8fdFbjvPw8R74n1oiY 

/home/icb/leon.stadelmann/miniconda3/envs/eot_benchmark/lib/python3.9/site-packages/eot_benchmark-1.0-py3.9.egg/eot_benchmark/gaussian_mixture_benchmark.py:196: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mo

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1HNXbrkozARbz4r8fdFbjvPw8R74n1oiY 

/home/icb/leon.stadelmann/miniconda3/envs/eot_benchmark/lib/python3.9/site-packages/eot_benchmark-1.0-py3.9.egg/eot_benchmark/gaussian_mixture_benchmark.py:224: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mo

In [8]:
neural_net = MLP_vector_field(DIM, 1024, 1024, 1024, n_frequencies=1024)
ot_solver = ott.solvers.linear.sinkhorn.Sinkhorn()
solver_latent_to_data = (
    ott.solvers.linear.sinkhorn.Sinkhorn() if K_NOISE_PER_X > 1 else None
)
optimizer = optax.adamw(learning_rate=LR, weight_decay=1e-10)

In [9]:
genot = GENOT(
    neural_net,
    optimizer=optimizer,
    ot_solver=ot_solver,
    epsilon=EPS,
    input_dim=DIM,
    output_dim=DIM,
    iterations=iters,
    k_noise_per_x=K_NOISE_PER_X,
    solver_latent_to_data=solver_latent_to_data,
    latent_to_data_scale_cost="mean",
    seed=SEED,
)

In [10]:
s_sampler = Loader(input_sampler, BATCH_SIZE)
t_sampler = Loader(target_sampler, BATCH_SIZE)



bws = []
cond_bws = []

test_samples = get_test_input_samples(dim=DIM, device=f"cuda:{GPU_DEVICE}").cpu()
test_samples_repeated = np.repeat(test_samples[:, None, :], NUM_SAMPLES_cBVP, axis=1)

cpu_device = jax.devices('cpu')[0]

/home/icb/leon.stadelmann/miniconda3/envs/eot_benchmark/lib/python3.9/site-packages/eot_benchmark-1.0-py3.9.egg/eot_benchmark/gaussian_mixture_benchmark.py:241: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues relate

In [11]:
source_test = input_sampler.sample(100_000)
target_test = target_sampler.sample(100_000)

In [12]:
for i in range(1):
    genot(s_sampler, t_sampler, BATCH_SIZE, BATCH_SIZE)
    predicted = jax.vmap(lambda *args, **kwargs: genot.transport(*args, **kwargs)[0])(
        jnp.asarray(test_samples_repeated), seed=jnp.arange(NUM_SAMPLES_cBVP)
    )
    predicted_squeezed = jnp.squeeze(predicted)
    cond_bw = calculate_cond_bw(
        test_samples, torch.tensor(np.asarray(predicted_squeezed)), eps=EPS, dim=DIM
    )

    source_samples = np.asarray(input_sampler.sample(NUM_SAMPLES_BVP).cpu())
    predicted = genot.transport(jnp.asarray(source_samples))
    predictions = torch.tensor(np.asarray(predicted[0][0, ...]))
    target_samples = target_sampler.sample(NUM_SAMPLES_BVP)
    bw = compute_BW_UVP_by_gt_samples(
        predictions.cpu().numpy(), target_samples.cpu().numpy()
    )

    bws.append(bw)
    cond_bws.append(cond_bw)


/home/icb/leon.stadelmann/miniconda3/envs/eot_benchmark/lib/python3.9/site-packages/eot_benchmark-1.0-py3.9.egg/eot_benchmark/gaussian_mixture_benchmark.py:224: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues relat

In [13]:
print(bws)

[0.08126619453644386]


In [14]:
print(cond_bws)

[9.060897295874108]
